# Installs and Imports

In [ ]:
!pip install pandas transformers datasets scikit-learn torch transformers[torch] optuna

In [ ]:
import pandas as pd
import numpy as np
import optuna
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import (
    XLMRobertaForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    AutoTokenizer,
    TrainerCallback,
    EarlyStoppingCallback
)
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import KFold
from tqdm import tqdm

# Drive Mounting

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Dataset Loading

In [ ]:
annotated_and_pseudolabeled_path = '/content/drive/My Drive/Research/SentimentAnalysisDivorce/Dataset/ANNOTATED_AND_PSEUDOLABELED_DATA_01.xlsx'
cleaned_preprocessed_path = '/content/drive/My Drive/Research/SentimentAnalysisDivorce/Dataset/CLEANED_PREPROCESSED_DATA_05.xlsx'

In [ ]:
df_combined_nd = pd.read_excel(annotated_and_pseudolabeled_path, sheet_name='ND')
df_combined_dn = pd.read_excel(annotated_and_pseudolabeled_path, sheet_name='DN')

In [ ]:
model_path_nd = '/content/drive/My Drive/Research/SentimentAnalysisDivorce/Models/ND'
model_path_dn = '/content/drive/My Drive/Research/SentimentAnalysisDivorce/Models/DN'

# Helper Functions

In [ ]:
def preprocess_data(df, tokenizer, max_length=512):
    texts = df['text'].tolist()
    labels = df['numericalLabel'].astype(int).tolist()
    inputs = tokenizer(texts, truncation=True, padding=True, max_length=max_length, return_tensors="pt")
    inputs['labels'] = torch.tensor(labels)
    return inputs

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, inputs):
        self.inputs = inputs

    def __len__(self):
        return len(self.inputs['input_ids'])

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.inputs.items()}
        return item

In [ ]:
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(p.label_ids, preds, average='weighted')
    acc = accuracy_score(p.label_ids, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
class BestModelSaverCallback(TrainerCallback):
    def __init__(self, trainer, tokenizer, patience=10, min_delta=0.0, output_dir=None):
        self.trainer = trainer
        self.tokenizer = tokenizer
        self.patience = patience
        self.min_delta = min_delta
        self.output_dir = output_dir
        self.best_loss = None
        self.best_accuracy = None
        self.patience_counter = 0

    def on_evaluate(self, args, state, control, **kwargs):
        logs = kwargs.get("metrics", {})
        eval_loss = logs.get("eval_loss")
        eval_accuracy = logs.get("eval_accuracy")

        if eval_loss is not None and eval_accuracy is not None:
            if self.best_loss is None or eval_loss < self.best_loss - self.min_delta:
                self.best_loss = eval_loss
                self.best_accuracy = eval_accuracy
                self.patience_counter = 0

                if self.output_dir:
                    self.trainer.model.save_pretrained(self.output_dir)
                    self.tokenizer.save_pretrained(self.output_dir)
            else:
                self.patience_counter += 1
                if self.patience_counter > self.patience:
                    control.should_training_stop = True

In [ ]:
tokenizer_xlm = AutoTokenizer.from_pretrained("xlm-roberta-base")

inputs_nd = preprocess_data(df_combined_nd, tokenizer_xlm)
inputs_dn = preprocess_data(df_combined_dn, tokenizer_xlm)

dataset_nd = CustomDataset(inputs_nd)
dataset_dn = CustomDataset(inputs_dn)

# Hyperparameter Tuning

In [ ]:
def objective(trial, train_dataset, model_path, category_name):
    # Define hyperparameters to tune
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    num_train_epochs = trial.suggest_int("num_train_epochs", 2, 5)
    per_device_train_batch_size = trial.suggest_categorical("per_device_train_batch_size", [8, 16, 32])
    gradient_accumulation_steps = trial.suggest_int("gradient_accumulation_steps", 1, 4)
    weight_decay = trial.suggest_float("weight_decay", 0, 0.3)
    warmup_steps = trial.suggest_int("warmup_steps", 0, 500)

    training_args = TrainingArguments(
        output_dir=f'/content/drive/My Drive/Research/SentimentAnalysisDivorce/Models/HyperparameterTuned/{category_name}',
        evaluation_strategy='epoch',
        save_strategy='epoch',
        learning_rate=learning_rate,
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=16,
        num_train_epochs=num_train_epochs,
        weight_decay=weight_decay,
        warmup_steps=warmup_steps,
        gradient_accumulation_steps=gradient_accumulation_steps,
        load_best_model_at_end=True,
        metric_for_best_model='eval_loss',
        greater_is_better=False,
        fp16=True,  # Use mixed precision training
    )

    # Initialize K-fold cross-validator
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    # Perform K-Fold cross-validation
    losses = []
    for fold, (train_index, eval_index) in enumerate(kf.split(train_dataset)):
        print(f"Starting fold {fold+1}...")
        train_subset = torch.utils.data.Subset(train_dataset, train_index)
        eval_subset = torch.utils.data.Subset(train_dataset, eval_index)

        model = XLMRobertaForSequenceClassification.from_pretrained(model_path, num_labels=7)
        data_collator = DataCollatorWithPadding(tokenizer=tokenizer_xlm)

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_subset,
            eval_dataset=eval_subset,
            data_collator=data_collator,
            compute_metrics=compute_metrics,
            callbacks=[
                EarlyStoppingCallback(early_stopping_patience=3),
            ]
        )

        trainer.train()
        eval_results = trainer.evaluate()
        losses.append(eval_results['eval_loss'])

        # Clear cache
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()

    # Return the average loss across all folds
    return np.mean(losses)

In [ ]:
study_nd = optuna.create_study(direction='minimize')
study_dn = optuna.create_study(direction='minimize')

In [14]:
model_path = model_path_nd
category_name = 'ND'
study_nd.optimize(lambda trial: objective(trial, dataset_nd, model_path, category_name), n_trials=3)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Starting fold 1...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.123380,0.968277,0.968163,0.968866,0.968277
2,No log,0.097493,0.974766,0.974819,0.975183,0.974766


Starting fold 2...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.086527,0.972603,0.972502,0.972518,0.972603
2,No log,0.056578,0.982696,0.982886,0.983664,0.982696


...

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
0,No log,0.043887,0.986291,0.986327,0.986846,0.986291
1,No log,0.042417,0.989177,0.989197,0.989258,0.989177


Starting fold 5...


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
0,No log,0.028068,0.990620,0.990629,0.990652,0.990620
1,No log,0.034822,0.986291,0.986316,0.986396,0.986291


[I 2024-07-09 09:01:54,296] Trial 2 finished with value: 0.0700259231030941 and parameters: {'learning_rate': 3.275467105788013e-05, 'num_train_epochs': 2, 'per_device_train_batch_size': 8, 'gradient_accumulation_steps': 4, 'weight_decay': 0.29144429794914223, 'warmup_steps': 236}. Best is trial 0 with value: 0.056785577535629274.


In [15]:
model_path = model_path_dn
category_name = 'DN'
study_dn.optimize(lambda trial: objective(trial, dataset_dn, model_path, category_name), n_trials=3)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Starting fold 1...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
0,No log,0.217241,0.939732,0.940631,0.944527,0.939732
1,No log,0.199210,0.939732,0.940654,0.944307,0.939732


Starting fold 2...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
0,No log,0.262886,0.928571,0.928251,0.930227,0.928571
1,No log,0.214142,0.944196,0.944214,0.946434,0.944196


...

Starting fold 4...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
0,No log,0.221130,0.957589,0.957749,0.958454,0.957589
1,No log,0.211238,0.957589,0.957384,0.958031,0.957589
3,No log,0.201132,0.959821,0.959766,0.960572,0.959821


Starting fold 5...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
0,No log,0.191163,0.955357,0.955279,0.955892,0.955357
1,No log,0.199166,0.950893,0.950897,0.951590,0.950893
3,No log,0.216001,0.948661,0.947704,0.948797,0.948661


[I 2024-07-09 09:49:36,360] Trial 2 finished with value: 0.19312008917331697 and parameters: {'learning_rate': 1.1738603684059503e-05, 'num_train_epochs': 4, 'per_device_train_batch_size': 8, 'gradient_accumulation_steps': 3, 'weight_decay': 0.12441406961535009, 'warmup_steps': 379}. Best is trial 2 with value: 0.19312008917331697.


In [17]:
best_trial_nd = study_nd.best_trial

In [18]:
best_trial_dn = study_dn.best_trial

In [19]:
print(f"Best trial ND: Value: {best_trial_nd.value}, Params: {best_trial_nd.params}")
print(f"Best trial DN: Value: {best_trial_dn.value}, Params: {best_trial_dn.params}")

Best trial ND: Value: 0.056785577535629274, Params: {'learning_rate': 2.9912691559155184e-05, 'num_train_epochs': 2, 'per_device_train_batch_size': 32, 'gradient_accumulation_steps': 1, 'weight_decay': 0.12046589013002564, 'warmup_steps': 131}
Best trial DN: Value: 0.19312008917331697, Params: {'learning_rate': 1.1738603684059503e-05, 'num_train_epochs': 4, 'per_device_train_batch_size': 8, 'gradient_accumulation_steps': 3, 'weight_decay': 0.12441406961535009, 'warmup_steps': 379}


# Final Training

In [ ]:
def train_with_best_params(best_params, category_name, model_path, train_dataset):
    training_args = TrainingArguments(
        output_dir=f'/content/drive/My Drive/Research/SentimentAnalysisDivorce/Models/{category_name}',
        evaluation_strategy='epoch',
        save_strategy='no',
        learning_rate=best_params['learning_rate'],
        per_device_train_batch_size=best_params['per_device_train_batch_size'],
        per_device_eval_batch_size=16,
        num_train_epochs=best_params['num_train_epochs'],
        weight_decay=0.01,
        gradient_accumulation_steps=best_params['gradient_accumulation_steps'],
        load_best_model_at_end=True,
        metric_for_best_model='eval_loss',
        greater_is_better=False,
        fp16=True,  # Use mixed precision training
    )

    # Initialize K-fold cross-validator
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    # Perform K-Fold cross-validation and train final model
    for fold, (train_index, eval_index) in enumerate(kf.split(train_dataset)):
        print(f"Starting fold {fold+1}...")
        train_subset = torch.utils.data.Subset(train_dataset, train_index)
        eval_subset = torch.utils.data.Subset(train_dataset, eval_index)

        model = XLMRobertaForSequenceClassification.from_pretrained(model_path, num_labels=7)
        data_collator = DataCollatorWithPadding(tokenizer=tokenizer_xlm)

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_subset,
            eval_dataset=eval_subset,
            data_collator=data_collator,
            compute_metrics=compute_metrics,
            callbacks=[BestModelSaverCallback(
                trainer=trainer,
                tokenizer=tokenizer_xlm,
                patience=3,
                output_dir=training_args.output_dir
            )]
        )

        for epoch in tqdm(range(training_args.num_train_epochs), desc=f"Training fold {fold+1}"):
            trainer.train()
            # Clear cache
            torch.cuda.empty_cache()
            torch.cuda.ipc_collect()